# Exploratory Data Analysis + Preprocessing

## Libraries

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import textwrap
from IPython.display import display

from typing import List, Dict, Tuple, Optional

## Read Database

In [2]:
## Connect Database
db_file = "n1_data_ops_challenge.db"
connection = sqlite3.connect(db_file)
cur = connection.cursor()

## Fetch table
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]    
print(f"Tables: {table_list}")

Tables: [('model_scores_by_zip',), ('roster_1',), ('roster_2',), ('roster_3',), ('roster_4',), ('roster_5',)]


In [3]:
## Load data into Pandas
roster_1 = pd.read_sql_query("SELECT * FROM roster_1;", connection)
roster_2 = pd.read_sql_query("SELECT * FROM roster_2;", connection)
roster_3 = pd.read_sql_query("SELECT * FROM roster_3;", connection)
roster_4 = pd.read_sql_query("SELECT * FROM roster_4;", connection)
roster_5 = pd.read_sql_query("SELECT * FROM roster_5;", connection)
model_scores = pd.read_sql_query("SELECT * FROM model_scores_by_zip;", connection)

## Table Exploration

In [4]:
## model_scores insight
model_scores.describe()

,zcta,state_code,neighborhood_stress_score,algorex_sdoh_composite_score,social_isolation_score,transportation_access_score,food_access_score,unstable_housing_score,state_govt_assistance,homeless_indicator,derived_indicator
count,1760.000000,1760.0,1760.000000,1760.000000,1760.000000,1760.000000,1760.000000,1760.000000,1760.000000,1760.000000,1760.000000
mean,93662.508523,6.0,-0.062892,6.427369,3.071813,4.238028,3.159642,2.638472,0.586636,7.162500,0.094886
std,1816.789712,0.0,0.690468,0.461369,0.803491,0.589008,0.801018,1.075871,0.541709,3.006486,0.293141
min,90001.000000,6.0,-1.410000,4.300000,0.000000,2.710000,0.280000,0.200000,0.000000,2.000000,0.000000
25%,92257.750000,6.0,-0.550000,6.117500,2.450000,3.880000,2.730000,1.970000,0.250000,4.000000,0.000000
50%,93656.500000,6.0,-0.210000,6.370000,2.920000,4.180000,3.210000,2.610000,0.440000,8.000000,0.000000
75%,95380.500000,6.0,0.290000,6.670000,3.600000,4.490000,3.700000,3.090000,0.770000,10.000000,0.000000
max,96161.000000,6.0,6.120000,8.770000,8.330000,8.750000,6.900000,7.880000,7.500000,10.000000,1.000000


In [5]:
## model_scores samples
model_scores.head(5)

,zcta,state_code,state name,neighborhood_stress_score,algorex_sdoh_composite_score,social_isolation_score,transportation_access_score,food_access_score,unstable_housing_score,state_govt_assistance,homeless_indicator,derived_indicator
0,94720,6.0,California,-0.31,6.19,2.37,3.89,3.06,2.03,0.59,10.0,1
1,95675,6.0,California,-0.31,6.18,3.75,4.42,3.90,2.76,0.33,6.0,1
2,95699,6.0,California,-0.31,6.18,3.75,4.42,3.90,2.76,0.33,6.0,1
3,95930,6.0,California,0.33,6.00,3.47,3.95,3.01,3.05,0.76,10.0,1
4,95941,6.0,California,0.33,6.00,3.47,3.95,3.01,3.05,0.76,10.0,1


In [6]:
## Roster data exploration
data = [roster_1, roster_2, roster_3, roster_4, roster_5]
print("Column Breakdown:")
for i in range(len(data)):
    label = f"Roster {i+1}"
    columns = " || ".join(sorted(list(data[i].columns)))
    wrapped = textwrap.fill(columns, width=100, subsequent_indent="\t\t")
    print(f"\t{label}: {wrapped}")

Column Breakdown:
	Roster 1: Age || City || Dob || First_Name || Gender || Last_Name || Person_Id || State || Street_Address ||
		Zip || eligibility_end_date || eligibility_start_date || payer
	Roster 2: Age || City || Dob || First_Name || Gender || Last_Name || Person_Id || State || Street_Address ||
		Zip || eligibility_end_date || eligibility_start_date || payer
	Roster 3: Age || City || Dob || First_Name || Gender || Last_Name || Person_Id || State || Street_Address ||
		Zip || eligibility_end_date || eligibility_start_date || payer
	Roster 4: Age || City || Dob || First_Name || Gender || Last_Name || Person_Id || State || Street_Address ||
		Zip || eligibility_end_date || eligibility_start_date || payer
	Roster 5: Age || City || Dob || First_Name || Gender || Last_Name || Person_Id || State || Street_Address ||
		Zip || eligibility_end_date || eligibility_start_date || payer


**All rosters have the same column schema.**

In [7]:
## Null Value Checker
print("Checking for Null values...")
for i in range(len(data)):
    label = f"Roster {i+1}"

    df = data[i]
    null_count = df.isna().sum().sum()
    total_count = df.count().sum()
    print(f"\t{label} --> {(null_count / total_count) * 100:.2f}% Null Values || {df.count().unique()[0]} Rows")

Checking for Null values...
	Roster 1 --> 0.00% Null Values || 23659 Rows
	Roster 2 --> 0.00% Null Values || 23392 Rows
	Roster 3 --> 0.00% Null Values || 34951 Rows
	Roster 4 --> 0.00% Null Values || 22900 Rows
	Roster 5 --> 0.00% Null Values || 37403 Rows


**No null value for each roster dataframe.**

### Sample Data for Roster Datasets

In [8]:
display(roster_1.head(1))
display(roster_2.head(1))
display(roster_3.head(1))
display(roster_4.head(1))
display(roster_5.head(1))

,Person_Id,First_Name,Last_Name,Dob,Age,Gender,Street_Address,State,City,Zip,eligibility_start_date,eligibility_end_date,payer
0,15340001,Daniel,Smith,2017-04-27,5,Male,1505 Alvarez Spur Suite 902,California,Lake Sharonburgh,93546,2021-08-01,2021-11-01,Madv


,Person_Id,First_Name,Last_Name,Dob,Age,Gender,Street_Address,State,City,Zip,eligibility_start_date,eligibility_end_date,payer
0,15340005,Maritza,Castellana,02/09/1979,43,Female,4097 Johnny Road,California,East Carolyntown,93206,10/01/2021,02/01/2023,Madv


,Person_Id,First_Name,Last_Name,Dob,Age,Gender,Street_Address,State,City,Zip,eligibility_start_date,eligibility_end_date,payer
0,15340053,Nathaniel,Sharkey,1925-01-26,97,Male,92776 Charles Lights Suite 296,California,East Veronica,95461,2021-12-01,2022-01-01,Mdcd


,Person_Id,First_Name,Last_Name,Dob,Age,Gender,Street_Address,State,City,Zip,eligibility_start_date,eligibility_end_date,payer
0,15340034,Amber,Smith,2000-02-01,22,Female,404 Gardner Pike Suite 348,CA,North Jefferyport,95620,2022-02-01,2022-06-01,Mdcd


,Person_Id,First_Name,Last_Name,Gender,Dob,Age,Street_Address,State,City,Zip,payer,eligibility_start_date,eligibility_end_date
0,15340012,Billy,Pacifico,Male,1989-04-03,33,6176 Nicholas Turnpike Apt. 850,California,West Dorothyburgh,90047,Mdcd,2022-04-01,2026-03-01


#### Data Type Validation
* Order of columns for `roster_5` is different from the other dataframes
  * Could be solved easily when merging - field data are still tied to column name
* Date type for `roster_2` is different from the other dataframes
  * Majority date format: `YYYY-MM-DD` --> Need extra parsing on `roster_2` before merging data
* State column for `roster_4` uses abbrieviation - different from the other dataframes
  * Parsing state abbreviation into full name solves the confusion

## Preprocessing

### Create `std_member_info` Table in Database

In [9]:
## Parsing function for all dates found in dataframe
def parse_date(
    data: pd.DataFrame, 
    input_format=None, 
    output_format="%Y-%m-%d", 
    errors="coerce", 
    inplace=False) -> Optional[pd.DataFrame]:

    modified_data = data if inplace else data.copy()

    for col in modified_data.columns:

        try:
            parser = pd.to_datetime(modified_data[col], format=input_format, errors=errors)
            if parser.notna().sum() > 0:
                modified_data[col] = parser.dt.strftime(output_format)
        except Exception as e:
            pass 

    return modified_data if not inplace else None

In [10]:
## Parsing function for state abbreviations

state_abbrev_to_name = { ## sourced from the internet
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

def parse_state(
    data: pd.Series,
    state_col_name: str,
    inplace=False
) -> Optional[pd.DataFrame]:

    modified_data = data if inplace else data.copy()
    try:
        modified_data[state_col_name] = modified_data[state_col_name].map(lambda s: state_abbrev_to_name[s])
    except Exception as e:
        print("No data matching column name found.")

    return modified_data if not inplace else None


In [11]:
## Parse dates in `roster_2` dataframe
parse_date(
    data=roster_2, 
    input_format="%m/%d/%Y",
    inplace=True
)
display(roster_2.head(1))

,Person_Id,First_Name,Last_Name,Dob,Age,Gender,Street_Address,State,City,Zip,eligibility_start_date,eligibility_end_date,payer
0,15340005,Maritza,Castellana,1979-02-09,43,Female,4097 Johnny Road,California,East Carolyntown,93206,2021-10-01,2023-02-01,Madv


In [12]:
## Parse states in `roster_4` dataframe
parse_state(
    data=roster_4, 
    state_col_name="State",
    inplace=True
)
display(roster_4.head(1))

,Person_Id,First_Name,Last_Name,Dob,Age,Gender,Street_Address,State,City,Zip,eligibility_start_date,eligibility_end_date,payer
0,15340034,Amber,Smith,2000-02-01,22,Female,404 Gardner Pike Suite 348,California,North Jefferyport,95620,2022-02-01,2022-06-01,Mdcd


In [13]:
## Create DataFrame without duplicates
roster_data = pd.concat(data, ignore_index=True).drop_duplicates()
roster_data

,Person_Id,First_Name,Last_Name,Dob,Age,Gender,Street_Address,State,City,Zip,eligibility_start_date,eligibility_end_date,payer
0,15340001,Daniel,Smith,2017-04-27,5,Male,1505 Alvarez Spur Suite 902,California,Lake Sharonburgh,93546,2021-08-01,2021-11-01,Madv
1,15340006,Todd,Austin,1934-01-06,88,Male,4731 Howe Ridge,California,New Rachel,95451,2021-08-01,2023-08-01,Madv
2,15340022,Leroy,Wilson,1960-09-20,62,Male,9710 Brianna Trail Apt. 145,California,Port Meredith,92222,2021-08-01,2024-01-01,Mdcd
3,15340042,Monica,Elmquist,1981-09-02,41,Female,47630 Sampson Throughway Suite 673,California,North Desireetown,95471,2021-08-01,2025-10-01,Mdcd
4,15340052,Betty,Read,1977-09-23,45,Female,78146 Angelica Lights Suite 526,California,Williambury,95018,2021-08-01,2024-08-01,Madv
...,...,...,...,...,...,...,...,...,...,...,...,...,...
128525,15539946,John,Koslow,1956-05-29,66,Male,7193 Jones Springs,California,Davidton,94965,2022-05-01,2025-09-01,Mdcd
128526,15539954,Lena,Wade,1993-03-31,29,Female,4827 Dawn Plaza Suite 699,California,Lake Amyland,95917,2022-05-01,2022-07-01,Madv
128527,15539956,Velma,Roberts,1946-12-05,76,Female,696 Martinez Glens Apt. 644,California,East Marcusborough,95521,2022-05-01,2025-11-01,Mdcd
128528,15539980,Antonio,Necaise,1943-09-02,79,Male,74800 Michele Brook,California,Matthewschester,93442,2022-05-01,2025-06-01,Mdcd


**There are 117685 rows of member data.**

In [15]:
## Export Data As Table in DB + Check if member_id is unique
member_info_data = roster_data.rename(columns={
    "Person_Id": "member_id", 
    "First_Name": "member_first_name",
    "Last_Name": "member_last_name",
    "Dob": "date_of_birth",
    "Zip": "zip_code",
    "City": "city",
    "State": "state",
    "Street_Address": "main_address",
}).drop(columns=["Age", "Gender"])
print(f"Count unique member ID: {len(member_info_data["member_id"].unique())}")

Count unique member ID: 117685


`Unique member ID (117685) != Original dataset size (117685)`

**No duplicate is found now. All member IDs are unique and belong to one specific member.**

In [16]:
## Check final parsed data
member_info_data.head(5)

,member_id,member_first_name,member_last_name,date_of_birth,main_address,state,city,zip_code,eligibility_start_date,eligibility_end_date,payer
0,15340001,Daniel,Smith,2017-04-27,1505 Alvarez Spur Suite 902,California,Lake Sharonburgh,93546,2021-08-01,2021-11-01,Madv
1,15340006,Todd,Austin,1934-01-06,4731 Howe Ridge,California,New Rachel,95451,2021-08-01,2023-08-01,Madv
2,15340022,Leroy,Wilson,1960-09-20,9710 Brianna Trail Apt. 145,California,Port Meredith,92222,2021-08-01,2024-01-01,Mdcd
3,15340042,Monica,Elmquist,1981-09-02,47630 Sampson Throughway Suite 673,California,North Desireetown,95471,2021-08-01,2025-10-01,Mdcd
4,15340052,Betty,Read,1977-09-23,78146 Angelica Lights Suite 526,California,Williambury,95018,2021-08-01,2024-08-01,Madv


In [20]:
member_info_data.describe()

,member_id,member_first_name,member_last_name,date_of_birth,main_address,state,city,zip_code,eligibility_start_date,eligibility_end_date,payer
count,117685,117685,117685,117685,117685,117685,117685,117685,117685,117685,117685
unique,117685,4222,22929,35077,117683,1,41508,1760,10,58,2
top,15340001,John,Smith,2003-06-24,914 William Walk,California,West Michael,92887,2021-12-01,2023-01-01,Mdcd
freq,1,2272,1294,13,2,117685,107,97,12070,2359,73653


In [22]:
member_info_data.payer.unique()

array(['Madv', 'Mdcd'], dtype=object)

**Observations**
* Only one state -> *California*
* Only 2 payer types -> *Madv*, *Mdcd*

## Extra Considerations \[Parsing Unseen Data\]

| Column Name |                      Anticipated Issues                      |
| ----------- | ------------------------------------------------------------ |
|  member_id  | <ul><li>Already existed in the dataset -> Identifier column: values have to be unique</li><li>(Assume `member_id` requires number only) Consist of only numbers</li></ul> |
|  member_first_name | <ul><li>**NO Punctuation or Number**</li></ul> |
|  member_last_name | <ul><li>**NO Punctuation or Number**</li></ul> |
|  date_of_birth | <ul><li style="color:gray">Date format follows the defaults: `%Y-%m-%d`</li><li>Too long ago -> Potentially invalid</li></ul> |
|  main_address | <ul><li>(Assume data encapsulates only the US) Follows US address format</li></ul> |
|  state \[category\] | <ul><li>**NO Punctuation or Number**</li><li>Matching one of the 51 state names</li><li style="color:gray">Parse abbreviations</li></ul> |
|  city | <ul><li>**NO Punctuation or Number**</li></ul> |
|  zip_code | <ul><li>Consist of 5 digits</li><li>Consist of numbers only</li></ul> |
|  eligibility_start_date | <ul><li style="color:gray">Date format follows the defaults: `%Y-%m-%d`</li></ul> |
|  eligibility_end_date | <ul><li style="color:gray">Date format follows the defaults: `%Y-%m-%d`</li></ul> |
|  payer \[category\] | <ul><li>(Assume there will be new payer category) Allow new payer type</li></ul> |